In [1]:
import open3d as o3d

bin_path = "/home/jk/ros2_test/src/ros2_orb_slam3/colmap_output/iteration0/final/sparse/0/points3D.bin"



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
import struct
import numpy as np

def read_points3D_bin(bin_path):
    points3D = {}
    with open(bin_path, "rb") as f:
        while True:
            binary_data = f.read(43 * 8 + 1)
            if not binary_data:
                break
            unpacked = struct.unpack("<QdddBBBdddddd", binary_data)
            point_id = unpacked[0]
            xyz = np.array(unpacked[1:4])
            rgb = np.array(unpacked[4:7], dtype=np.uint8)
            # 나머지: error, track 정보는 생략 가능
            points3D[point_id] = (xyz, rgb)
    return points3D

In [1]:
import numpy as np
import open3d as o3d

def read_points3D_txt(txt_path):
    points3D = {}
    with open(txt_path, "r") as f:
        for line in f:
            if line.startswith("#") or line.strip() == "":
                continue
            elems = line.strip().split()
            point_id = int(elems[0])
            xyz = np.array([float(elems[1]), float(elems[2]), float(elems[3])])
            rgb = np.array([int(elems[4]), int(elems[5]), int(elems[6])], dtype=np.uint8)
            points3D[point_id] = (xyz, rgb)
    return points3D

def draw_point_cloud_with_custom_size(pcd, point_size=1.0):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pcd)

    # 렌더링 옵션 설정
    render_option = vis.get_render_option()
    render_option.point_size = point_size  # 포인트 크기 설정 (기본은 1.0)
    render_option.background_color = np.array([0, 0, 0])  # 배경색 (선택사항)

    vis.run()
    vis.destroy_window()

def visualize_colmap_points(txt_path):
    points3D = read_points3D_txt(txt_path)

    xyzs = []
    rgbs = []

    for xyz, rgb in points3D.values():
        xyzs.append(xyz)
        rgbs.append(rgb / 255.0)  # Open3D는 [0,1] 범위로 색상 요구

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np.array(xyzs))
    pcd.colors = o3d.utility.Vector3dVector(np.array(rgbs))

    draw_point_cloud_with_custom_size(pcd, point_size=0.5)

# 실행
txt_path = "/home/jk/ros2_test/src/ros2_orb_slam3/colmap_output/iteration0/final/sparse/0/points3D.txt"
visualize_colmap_points(txt_path)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
